In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

In [2]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, AveragePooling2D, Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
import imutils
from imutils import paths

**Importing our Dataset**

In [3]:
dataset = r"dataset"

*setting image paths to variables with all images at once*

In [4]:
imagePaths = list(paths.list_images(dataset))

**Extracting Data and labels from dataset**

In [5]:
labels = []
data = []

for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i, target_size = (224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

In [6]:
labels = np.array(labels)
data = np.array(data, dtype = "float32")

In [7]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

**Splitting Our Data into train Data and split Data**

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 10, stratify = labels)

Here we will use model called MobileNet, therefore we will download it below. This model is very light weight and provide similar efficiency as VGG. We will store it in variable called baseModel

In [9]:
baseModel = MobileNetV2(weights = 'imagenet', include_top = False, input_tensor = Input(shape = (224,224,3)))

**Lets make layers of our baseModel as Non Trainable because, We need its pretrained value for detection, we will ony train headModel**

In [10]:
for layer in baseModel.layers:
    layer.trainable = False

**Let us make our head of baseModel that will be trainable**

In [11]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (7,7))(headModel)
headModel = Flatten(name = "Flatten")(headModel)
headModel = Dense(128, activation = 'relu')(headModel)
headModel = Dropout(0.4)(headModel)
headModel = Dense(2, activation = 'softmax')(headModel)

In [12]:
model = Model(inputs = baseModel.input, outputs = headModel)

In [13]:
#Image Data Generation
aug = ImageDataGenerator(rotation_range = 50, zoom_range = 0.2, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True,vertical_flip = True, fill_mode = "nearest")

**Training Our Model**

In [14]:
learning_rate = 0.001
Epochs = 36
BS = 12 #batch size

opt = tf.keras.optimizers.Adam(lr = learning_rate, decay = learning_rate/Epochs)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [15]:
model.fit(aug.flow(x_train, y_train, batch_size = BS),
         steps_per_epoch = len(x_train)//BS,
         validation_data = (x_test, y_test),
         validation_steps = len(x_test)//BS,
         epochs = Epochs
         )

Epoch 1/36
91/91 [==============================] - 47s 520ms/step - loss: 0.2231 - accuracy: 0.9136 - val_loss: 0.0217 - val_accuracy: 0.9928
Epoch 2/36
91/91 [==============================] - 46s 502ms/step - loss: 0.0479 - accuracy: 0.9825 - val_loss: 0.0142 - val_accuracy: 0.9928
Epoch 3/36
91/91 [==============================] - 46s 511ms/step - loss: 0.0435 - accuracy: 0.9825 - val_loss: 0.0107 - val_accuracy: 0.9964
Epoch 4/36
91/91 [==============================] - 46s 506ms/step - loss: 0.0631 - accuracy: 0.9825 - val_loss: 0.0159 - val_accuracy: 0.9928
Epoch 5/36
91/91 [==============================] - 47s 512ms/step - loss: 0.0358 - accuracy: 0.9853 - val_loss: 0.0132 - val_accuracy: 0.9964
Epoch 6/36
91/91 [==============================] - 46s 508ms/step - loss: 0.0350 - accuracy: 0.9890 - val_loss: 0.0165 - val_accuracy: 0.9891
Epoch 7/36
91/91 [==============================] - 46s 506ms/step - loss: 0.0254 - accuracy: 0.9908 - val_loss: 0.0237 - val_accuracy: 0.9891

In [16]:
model.save(r"facemask.model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: facemask.model/assets


In [17]:
from sklearn.metrics import classification_report

preds = model.predict(x_test, batch_size = BS)
preds = np.argmax(preds, axis = 1)


In [18]:
print(classification_report(y_test.argmax(axis = 1),preds, target_names = lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       138
without_mask       0.99      0.99      0.99       138

    accuracy                           0.99       276
   macro avg       0.99      0.99      0.99       276
weighted avg       0.99      0.99      0.99       276

